In [1]:
'''TODO
known bugs:
if you have port that gets 1 part of a bus, like:
    .vin(i_ffe_cm2_rto[8])
the netlister will not print this port
'''

'TODO\nknown bugs:\nif you have port that gets 1 part of a bus, like:\n    .vin(i_ffe_cm2_rto[8])\nthe netlister will not print this port\n'

In [2]:
import ipywidgets
import pandas as pd
import os
import hashlib
pd.set_option('display.max_colwidth',1000)

parser_output=None
df=None
verilog_output_folder='/nfs/iil/disks/ams_regruns/dkl/users/lisrael1/msv/temp_del/verilog_helper/'
pattern=r'son - (?P<son_net>[\w\d_]+)@(?P<son_block>[\w\d_]+) \((?P<son_type>[\w_-]*):.*:(?P<son_direction>[\w_-]*)\) father - (?P<father_net>[\w\d_\[\]]+)@(?P<father_block>[\w\d_]+) \((?P<father_type>[\w_-]*):.*father: (?P<father_file>.*) son: (?P<son_file>.*)'

In [3]:
verilog_folder_widget=ipywidgets.Text(value=r'/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/brk_gen2/4.brk_gen2_0p8/0.original/1',placeholder='verilog folder full path',
                                      description='verilog folder:', 
                                      layout=ipywidgets.Layout(width='95%'),
                                      style=dict(description_width='initial'))
@ipywidgets.interact
def a(verilog_folder=verilog_folder_widget):
    global parser_output, df
    # it takes a lot of time to run, so we will save old results by the folder hash and reload this result at next time
    verilog_folder_hash=hashlib.md5(str.encode(verilog_folder)).hexdigest()
    parser_output_file=verilog_output_folder+'/perl_verilog_parser_output_'+verilog_folder_hash+'.txt'
    csv_output_file=verilog_output_folder+'/csv_verilog_parser_output_'+verilog_folder_hash+'.csv'

    exist=!ls $parser_output_file
    if exist[0].startswith('ls: cannot access '):
        print('running parser (may take 1 minute)')
        command='/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/tools/sv/print_all_modules_and_ports_connection.pl -f='+\
            verilog_folder+' 2>&1 |pv -l -N "lines rate at reading parser output"> '+\
            parser_output_file
        print('running command:\n\t'+command)
        !$command
    print('parsing file '+parser_output_file)
    parser_output=open(parser_output_file).read().split('\n')
    print('done parsing')
    df=pd.DataFrame(parser_output, columns=['original'])
    print('converting to csv')
    df=pd.concat([df.original.str.extract(pattern),df], axis=1)
    print('saving csv (may take a minute)')
    df.to_csv(csv_output_file)
    print('saved to '+csv_output_file)
    display(df.tail(10))
#         display(df.query('((son_block==@block) and (son_net==@net)) or ((father_block==@block) and (father_net==@net)) '))
    

interactive(children=(Text(value='/nfs/iil/disks/hdk_ws/ckt/lisrael1/lior_dir/bmods_development/brk_gen2/4.brk…

In [4]:
@ipywidgets.interact
def a(block='block name', net='net name'):
    global df
    if not df is None:
        block=block.replace(' ','')
        net=net.replace(' ','')
        display(df.query('((son_block==@block) and (son_net==@net)) or ((father_block==@block) and (father_net==@net)) '))     

interactive(children=(Text(value='block name', description='block'), Text(value='net name', description='net')…